In [1]:
import pandas as pd
import hashlib

In [2]:
md5 = lambda s: hashlib.md5(s).hexdigest()

In [3]:
#使用pandas读取excel文件
base_path = 'E:/gaohe_data/8个专题对话记录/'
output_all_path = '../data/weixin_8_title_data.txt'

In [4]:
__LABEL__ = "__label__"

__LABEL_UNK__ = __LABEL__ + "UNK"
__LABEL_MULTIPLE__ = __LABEL__ +  "multiple"

def process_service(table1, file_path, service_obj):
    
    def check_key(sentense):
        for k in service_obj._key:
            if sentense.find(k) != -1:
                return True

        return False

    def check_key_multiple(sentense):
        for k in service_obj._key_multiple:
            if sentense.find(k) != -1:
                return True

        return False
    
    def check_key_other(sentense):
        for k in service_obj._key_other:
            if sentense.find(k) != -1:
                return True

        return False
    
    _question = table1['Unnamed: 2']
    _classfilter = table1['Unnamed: 6']

    fw = open(file_path, "a", encoding="utf-8")

    texts_set = set()
    for i, (q, c) in enumerate(zip(_question, _classfilter)):
        if i <= 1:
            continue

        if (i < 20):
            print(str(i) + " " + q + " " + c)

        _md5_v = md5(q.encode('utf-8'))
        if _md5_v in texts_set:
            if (i < 20):
                print("texts_set: ", q)
            continue

        texts_set.add(_md5_v)

        _label = service_obj.get_label(str(c))
        if len(_label) > 0:
            if check_key(str(q)):
                #_label = service_obj.map_label[c]
                _label = _label
            elif check_key_multiple(str(q)):
                #print("aaaaa__label__multiple", q + " " + c)
                _label = __LABEL__ +  "multiple"
            else:
                #print("bbbb__label_UNK", q + " " + c)
                _label = __LABEL_UNK__
        else:
            if (c == "人机互动") and check_key_other(str(q)):
                #print("cccc_defalut_label", q + " " + c)
                _label = service_obj.defalut_label  
            else:
                #print("dddd__label_UNK", q + " " + c)
                _label = __LABEL_UNK__
                              
        if (i < 20):
            print(str(i) + " " + q + " " + _label)
            
        if _label != __LABEL_UNK__ and _label != __LABEL_MULTIPLE__:
            _s = ""
            for _c in q.strip():
                _s += _c + " "
                
            fw.write("{}{}\n".format(_s, _label))

    fw.close()
    
def _process(src_file_path, output_file_path, process_obj):
    _xls_file = pd.ExcelFile(src_file_path)
    _table = _xls_file.parse('对话记录表')
    
    # process_service(_table, output_file_path, process_obj)
    process_service(_table, output_all_path, process_obj)

In [5]:
class ShengYu:
    def __init__(self):
        
        self.name = "职工生育保险待遇"
        self.defalut_label = __LABEL__ + "职工生育保险待遇"

        self.map_label = {"职工生育保险待遇":self.defalut_label,
                    "失败问题回复":__LABEL_UNK__,
                    "此节点为根节点，无场景":__LABEL_UNK__
                    }

        # 身份证业务相关的 关键词
        self._key = ["生育", "保险", "产检"]
        
        # 可以多意图关键词
        self._key_multiple = ["报销", "补贴", "津贴"]
        
        # 人机互动   为了获取更多语料，可以不需要
        self._key_other = ["生育保险", "生育津贴", "产检报销"]

    def get_label(self, _class):
        _label_out = ""
        _len = len(self.name)
        if _class[0:_len] == self.name:
            _label_out = self.defalut_label
        else:
            if _class in self.map_label:
                _label_out = self.map_label[_class]

        return _label_out
        
        return ""

_process(base_path + '《职工生育保险待遇》对话记录.xls', "职工生育保险待遇_label.txt", ShengYu())

2 你好 人机互动
2 你好 __label__UNK
3 请问自离不发工资和离吗 失败问题回复
3 请问自离不发工资和离吗 __label__UNK
4 自离 失败问题回复
4 自离 __label__UNK
5 你好 人机互动
texts_set:  你好
6 生育津贴的时间是多少天？ 职工生育保险待遇-生育津贴
6 生育津贴的时间是多少天？ __label__职工生育保险待遇
7 买了居民医保，没有社保，能不能报销 失败问题回复
7 买了居民医保，没有社保，能不能报销 __label__multiple
8 生育津贴申请通过了什么时候发放 职工生育保险待遇-生育津贴
8 生育津贴申请通过了什么时候发放 __label__职工生育保险待遇
9 哦 此节点为根节点，无场景
9 哦 __label__UNK
10 男 此节点为根节点，无场景
10 男 __label__UNK
11 你好你是机器人家里小鸡可以吧，快点叫小鸡鸡 失败问题回复
11 你好你是机器人家里小鸡可以吧，快点叫小鸡鸡 __label__UNK
12 哇 此节点为根节点，无场景
12 哇 __label__UNK
13 生育二胎 失败问题回复
13 生育二胎 __label__UNK
14 哼 此节点为根节点，无场景
14 哼 __label__UNK
15 男职工是否有生育津贴 职工生育保险待遇-生育津贴
15 男职工是否有生育津贴 __label__职工生育保险待遇
16 生育报销的标准 失败问题回复
16 生育报销的标准 __label__UNK
17 生完小孩多久内要申请补贴 失败问题回复
17 生完小孩多久内要申请补贴 __label__multiple
18 我想问一下我的那个生育报销现在报到账了没有 失败问题回复
18 我想问一下我的那个生育报销现在报到账了没有 __label__UNK
19 在外省生产的费用可以拿回佛山报销吗 失败问题回复
19 在外省生产的费用可以拿回佛山报销吗 __label__multiple


In [6]:
class YinShiPasport:
    def __init__(self):
        
        self.name = "因私护照"
        self.defalut_label = __LABEL__ + "因私护照"

        self.map_label = {"因私护照":self.defalut_label,
                    "失败问题回复":__LABEL_UNK__,
                    "此节点为根节点，无场景":__LABEL_UNK__
                    }

        # 身份证业务相关的 关键词
        self._key = ["护照"]
        
        # 可以多意图关键词
        self._key_multiple = ["首次", "申领", "补办", "焕发"]
        
        # 人机互动   为了获取更多语料，可以不需要
        self._key_other = ["护照"]

    def get_label(self, _class):
        _label_out = ""
        _len = len(self.name)
        if _class[0:_len] == self.name:
            _label_out = self.defalut_label
        else:
            if _class in self.map_label:
                _label_out = self.map_label[_class]

        return _label_out
        
        return ""

_process(base_path + '《因私护照》对话记录.xls', "因私护照_label.txt", YinShiPasport())

2 公积金提取 此节点为根节点，无场景
2 公积金提取 __label__UNK
3 外省人有居住证可以办理吗 失败问题回复
3 外省人有居住证可以办理吗 __label__UNK
4 请问美国签证 失败问题回复
4 请问美国签证 __label__UNK
5 因私人护照 因私护照
5 因私人护照 __label__因私护照
6 广东外市户籍能否在佛山办理护照？ 因私护照
6 广东外市户籍能否在佛山办理护照？ __label__因私护照
7 外地户口护照多就可以办理 因私护照
7 外地户口护照多就可以办理 __label__因私护照
8 婴儿办理护照所需资料 因私护照
8 婴儿办理护照所需资料 __label__因私护照
9 您好 人机互动
9 您好 __label__UNK
10 如何办理护照 因私护照
10 如何办理护照 __label__因私护照
11 异地签证 失败问题回复
11 异地签证 __label__UNK
12 如何办护照 因私护照
12 如何办护照 __label__因私护照
13 没有居住证可以办理护照吗？ 因私护照
13 没有居住证可以办理护照吗？ __label__因私护照
14 你好 人机互动
14 你好 __label__UNK
15 多久 失败问题回复
15 多久 __label__UNK
16 商务签证 失败问题回复
16 商务签证 __label__UNK
17 出国旅游 失败问题回复
17 出国旅游 __label__UNK
18 周六可以申请吗 失败问题回复
18 周六可以申请吗 __label__UNK
19 你好 人机互动
texts_set:  你好


In [7]:
class YiLiaoBaoXiao:
    def __init__(self):
        
        self.name = "医疗报销待遇"
        self.defalut_label = __LABEL__ + "医疗报销待遇"

        self.map_label = {"医疗报销待遇":self.defalut_label,
                    "失败问题回复":__LABEL_UNK__,
                    "此节点为根节点，无场景":__LABEL_UNK__
                    }

        # 身份证业务相关的 关键词
        self._key = ["医疗", "医疗保险", "报销", "医保", "住院待遇", "门诊待遇"]
        
        # 可以多意图关键词
        self._key_multiple = ["报销", "保险", "待遇"]
        
        # 人机互动   为了获取更多语料，可以不需要
        self._key_other = ["医疗","住院待遇", "门诊待遇"]

    def get_label(self, _class):
        _label_out = ""
        _len = len(self.name)
        if _class[0:_len] == self.name:
            _label_out = self.defalut_label
        else:
            if _class in self.map_label:
                _label_out = self.map_label[_class]

        return _label_out
        
        return ""

_process(base_path + '《医疗报销待遇》对话记录.xls', "医疗报销待遇_label.txt", YiLiaoBaoXiao())

2 门诊 此节点为根节点，无场景
2 门诊 __label__UNK
3 报销 失败问题回复
3 报销 __label__UNK
4 呵呵 此节点为根节点，无场景
4 呵呵 __label__UNK
5 嗯嗯 失败问题回复
5 嗯嗯 __label__UNK
6 嘻嘻 此节点为根节点，无场景
6 嘻嘻 __label__UNK
7 高明区的门诊医保在禅城区如何使用 医疗报销待遇-高明区-门诊待遇（高明区）
7 高明区的门诊医保在禅城区如何使用 __label__医疗报销待遇
8 广州医保，在佛山住院是否可以吗？ 医疗报销待遇
8 广州医保，在佛山住院是否可以吗？ __label__医疗报销待遇
9 佛山社保、在广州中山第一附属人民医院生殖中心门诊看病，可以报销吗 医疗报销待遇-高明区-门诊待遇（高明区）
9 佛山社保、在广州中山第一附属人民医院生殖中心门诊看病，可以报销吗 __label__医疗报销待遇
10 你好 人机互动
10 你好 __label__UNK
11 便民服务在哪儿 失败问题回复
11 便民服务在哪儿 __label__UNK
12 你好，想了解一下社保停交了两个月，再从交的话，医疗保险从什么时候可以享受？ 医疗报销待遇
12 你好，想了解一下社保停交了两个月，再从交的话，医疗保险从什么时候可以享受？ __label__医疗报销待遇
13 随迁子女如何报销住院费用 失败问题回复
13 随迁子女如何报销住院费用 __label__UNK
14 111 失败问题回复
14 111 __label__UNK
15 2 此节点为根节点，无场景
15 2 __label__UNK
16 无发票能报销吗？ 失败问题回复
16 无发票能报销吗？ __label__UNK
17 怎么办理 失败问题回复
17 怎么办理 __label__UNK
18 补交社保，医疗从什么时候可以享受 失败问题回复
18 补交社保，医疗从什么时候可以享受 __label__UNK
19 三甲医院普通门诊能报销 医疗报销待遇
19 三甲医院普通门诊能报销 __label__医疗报销待遇


In [8]:
class ShebaoCard:
    def __init__(self):
        
        self.name = "社保卡办理及查询方式"
        self.defalut_label = __LABEL__ +  "社保卡办理及查询方式"

        self.map_label = {"社保卡办理及查询方式":self.defalut_label,
                    "失败问题回复":__LABEL_UNK__,
                    "此节点为根节点，无场景":__LABEL_UNK__
                    }

        # 身份证业务相关的 关键词
        self._key = ["社保卡"]
        
        # 可以多意图关键词
        self._key_multiple = ["办理", "查询"]
        
        # 人机互动   为了获取更多语料，可以不需要
        self._key_other = ["社保卡"]

    def get_label(self, _class):
        _label_out = ""
        _len = len(self.name)
        if _class[0:_len] == self.name:
            _label_out = self.defalut_label
        else:
            if _class in self.map_label:
                _label_out = self.map_label[_class]

        return _label_out
        
        return ""

_process(base_path + '《社保卡办理及查询方式》对话记录.xls', "社保卡办理及查询方式_label.txt", ShebaoCard())

2 广州户口佛山购买社保如何办理 失败问题回复
2 广州户口佛山购买社保如何办理 __label__multiple
3 广州户口佛山购买社保如何办理社保卡 此节点为根节点，无场景
3 广州户口佛山购买社保如何办理社保卡 __label__UNK
4 广州户口，佛山购买社保，如何办理社保卡 此节点为根节点，无场景
4 广州户口，佛山购买社保，如何办理社保卡 __label__UNK
5 员工是广州户口的，在佛山购买社保，如果要办理社保卡，如何操作 社保卡办理及查询方式
5 员工是广州户口的，在佛山购买社保，如果要办理社保卡，如何操作 __label__社保卡办理及查询方式
6 你会 失败问题回复
6 你会 __label__UNK
7 你好 人机互动
7 你好 __label__UNK
8 牛个鬼 失败问题回复
8 牛个鬼 __label__UNK
9 逆天后怕流量 失败问题回复
9 逆天后怕流量 __label__UNK
10 牛肉给句话 失败问题回复
10 牛肉给句话 __label__UNK
11 牛热狗虎 失败问题回复
11 牛热狗虎 __label__UNK
12 稳的社保卡 社保卡办理及查询方式
12 稳的社保卡 __label__社保卡办理及查询方式
13 哦豁破鼓跌社保卡 社保卡办理及查询方式
13 哦豁破鼓跌社保卡 __label__社保卡办理及查询方式
14 柔和虎社保卡 社保卡办理及查询方式
14 柔和虎社保卡 __label__社保卡办理及查询方式
15 社保卡准确准生证 此节点为根节点，无场景
15 社保卡准确准生证 __label__UNK
16  哦去基地社保卡 社保卡办理及查询方式
16  哦去基地社保卡 __label__社保卡办理及查询方式
17 申领居住证 失败问题回复
17 申领居住证 __label__UNK
18 顶层 社保卡办理及查询方式
18 顶层 __label__UNK
19 社保可以自己买吗 失败问题回复
19 社保可以自己买吗 __label__UNK


In [9]:
class ShebaoQuery:
    def __init__(self):
        
        self.name = "个人社保信息查询"
        self.defalut_label = __LABEL__ +  "个人社保信息查询"

        self.map_label = {"个人社保信息查询":self.defalut_label,
                    "个人社保信息查询-中国公民（港澳台除外）":__LABEL__ +  "个人社保信息查询-中国公民",
                    "个人社保信息查询-港澳台及外籍人员":__LABEL__ +  "个人社保信息查询-港澳台及外籍人员",
                    "失败问题回复":__LABEL_UNK__,
                    "此节点为根节点，无场景":__LABEL_UNK__
                    }

        # 身份证业务相关的 关键词
        self._key = ["社保", "参保", "入户积分查询"]
        
        # 可以多意图关键词
        self._key_multiple = ["社保卡", "查询", "咨询"]
        
        # 人机互动   为了获取更多语料，可以不需要
        self._key_other = ["社保信息查询","社保查询", 
                           "参保信息查询","参保查询",
                           "社保信息咨询","社保咨询", 
                           "参保信息咨询","参保咨询"]

    def get_label(self, _class):
        _label_out = ""
        _len = len(self.name)
        if _class[0:_len] == self.name:
            _label_out = self.defalut_label
        else:
            if _class in self.map_label:
                _label_out = self.map_label[_class]

        return _label_out
        
        return ""

_process(base_path + '《个人社保信息查询》对话记录.xls', "个人社保信息查询_label.txt", ShebaoQuery())

2 社保查询 个人社保信息查询
2 社保查询 __label__个人社保信息查询
3 我的社保金情况如何？ 失败问题回复
3 我的社保金情况如何？ __label__UNK
4 社保 个人社保信息查询
4 社保 __label__个人社保信息查询
5 我要怎样查社保啊 个人社保信息查询
5 我要怎样查社保啊 __label__个人社保信息查询
6 查社保卡余额 个人社保信息查询
6 查社保卡余额 __label__个人社保信息查询
7 人工 个人社保信息查询
7 人工 __label__UNK
8 我交了多少年社保 失败问题回复
8 我交了多少年社保 __label__UNK
9 是 失败问题回复
9 是 __label__UNK
10 查社保 个人社保信息查询
10 查社保 __label__个人社保信息查询
11 外地户口个人如何在佛山交社保 失败问题回复
11 外地户口个人如何在佛山交社保 __label__UNK
12 能否查我的社保状态 失败问题回复
12 能否查我的社保状态 __label__UNK
13 我的社保 个人社保信息查询
13 我的社保 __label__个人社保信息查询
14 社保余额 失败问题回复
14 社保余额 __label__UNK
15 我想查下我交了多长时间社保 失败问题回复
15 我想查下我交了多长时间社保 __label__UNK
16 社保卡17年的钱为什么没返 个人社保信息查询
16 社保卡17年的钱为什么没返 __label__个人社保信息查询
17 社保为何没有个人返还了 失败问题回复
17 社保为何没有个人返还了 __label__UNK
18 g 失败问题回复
18 g __label__UNK
19 社保查询 个人社保信息查询
texts_set:  社保查询


In [10]:
class Identify:
    def __init__(self):
        
        self.name = "身份证业务"
        self.defalut_label = __LABEL__ +  "身份证业务"

        self.map_label = {"身份证业务":self.defalut_label,
                    "身份证业务-首次申领":__LABEL__ + "身份证_首次申领",
                    "身份证业务-挂失":__LABEL__ + "身份证_挂失",
                    "身份证业务-遗失补办":__LABEL__ + "身份证_补办",
                    "身份证业务-到期、损坏换领":__LABEL__ + "身份证_换领",
                    "身份证业务-临时身份证":__LABEL__ + "身份证_临时身份证",
                    "身份证业务-自助机办证/领证":__LABEL__ + "身份证_自助机",
                    "身份证业务-预约途径":__LABEL__ + "身份证_预约途径",
                    "失败问题回复":__LABEL_UNK__,
                    "此节点为根节点，无场景":__LABEL_UNK__
                    }

        # 身份证业务相关的 关键词
        self._key = ["身份证", "婴儿入户", "新生儿入户"]
        
        # 可以多意图关键词
        self._key_multiple = ["首次申领", "申领", "挂失", "遗失补办", "到期", "损坏换领", "办证", "领证"]
        
        # 人机互动   为了获取更多语料，可以不需要
        self._key_other = []

    def get_label(self, _class):
        if _class in self.map_label:
            return self.map_label[_class]
        
        return ""

_process(base_path + '《身份证业务》对话记录.xls', "身份证_label.txt", Identify())

2 测试 失败问题回复
2 测试 __label__UNK
3 身份证 身份证业务
3 身份证 __label__身份证业务
4 女孩子速递社保卡 此节点为根节点，无场景
4 女孩子速递社保卡 __label__UNK
5 身份证 身份证业务
texts_set:  身份证
6 未成年人 失败问题回复
6 未成年人 __label__UNK
7 你好，我是乐从的，请问可以在禅城办身份证吗 身份证业务
7 你好，我是乐从的，请问可以在禅城办身份证吗 __label__身份证业务
8 你会些什么？ 身份证业务
8 你会些什么？ __label__UNK
9 你好啊，小机器人 人机互动
9 你好啊，小机器人 __label__UNK
10 谢谢 人机互动
10 谢谢 __label__UNK
11 哈哈 人机互动
11 哈哈 __label__UNK
12 你快乐吗 人机互动
12 你快乐吗 __label__UNK
13 有点傻 失败问题回复
13 有点傻 __label__UNK
14 请问，我的身份证丢了要怎么办？ 身份证业务
14 请问，我的身份证丢了要怎么办？ __label__身份证业务
15 身份证丢了要怎么办 身份证业务
15 身份证丢了要怎么办 __label__身份证业务
16 身份证丢失怎么办 身份证业务-挂失
16 身份证丢失怎么办 __label__身份证_挂失
17 身份证丢失了 身份证业务-挂失
17 身份证丢失了 __label__身份证_挂失
18 禅城地区身份证丢失 身份证业务-挂失
18 禅城地区身份证丢失 __label__身份证_挂失
19 身份证到期怎么办 身份证业务-到期、损坏换领
19 身份证到期怎么办 __label__身份证_换领


In [11]:
class HongKongAoMenPassport:
    def __init__(self):
        
        self.name = "港澳通行证"
        self.defalut_label = __LABEL__ + "港澳通行证"

        self.map_label = {"港澳通行证":self.defalut_label,
#                           "港澳通行证-因私业务":"__label__港澳通行证_因私业务",
#                           "港澳通行证-因私业务-换发港澳通行证":"__label__港澳通行证_因私业务_换发",
#                           "港澳通行证-因私业务-换发港澳通行证-本省外市户籍居民（换发）":"__label__港澳通行证_因私业务_外市_换发",
#                           "港澳通行证-因私业务-补办港澳通行证-暂住本市的华侨（补办）":"__label__港澳通行证_因私业务_补办",
#                           "港澳通行证-因私业务-首次申领港澳通行证":"__label__港澳通行证_因私业务",
#                           "港澳通行证-因私业务-首次申领港澳通行证-本省外市户籍居民（首次）":"__label__港澳通行证_因私业务_外市_首次",                          
#                           "港澳通行证-因私业务-首次申领港澳通行证-本市户籍居民（首次）":"__label__港澳通行证_因私业务_本市_换发",                          
                          "失败问题回复":__LABEL_UNK__,
                          "此节点为根节点，无场景":__LABEL_UNK__
                    }

        # 公积金提取业务相关的 关键词 （模糊，不需要精准）
        self._key = ["港澳", "通行证","签证","签注"]
        
        # 可以多意图关键词
        self._key_multiple = ["首次申领", "焕发", "补办"]
        
        # 人机互动   为了获取更多语料，可以不需要
        self._key_other = ["港澳通行证","通行证"]
        
    def get_label(self, _class):
        _label_out = ""
        _len = len(self.name)
        if _class[0:_len] == self.name:
            _label_out = self.defalut_label
        else:
            if _class in self.map_label:
                _label_out = self.map_label[_class]

        return _label_out
        
_process(base_path + '《港澳通行证》对话记录.xls', "港澳通行证_label.txt", HongKongAoMenPassport())

2 佛山市港澳通行证 港澳通行证
2 佛山市港澳通行证 __label__港澳通行证
3 签注一次过多久才能再签 此节点为根节点，无场景
3 签注一次过多久才能再签 __label__UNK
4 我要打官司 失败问题回复
4 我要打官司 __label__UNK
5 1 港澳通行证-因私业务-换发港澳通行证-本省外市户籍居民（换发）
5 1 __label__UNK
6 需要本人回户籍所在地办理吗？ 失败问题回复
6 需要本人回户籍所在地办理吗？ __label__UNK
7 地址 失败问题回复
7 地址 __label__UNK
8 你好 人机互动
8 你好 __label__UNK
9 外地户口能在佛山办理吗 失败问题回复
9 外地户口能在佛山办理吗 __label__UNK
10 怎样办证 港澳通行证
10 怎样办证 __label__UNK
11 请问，澳门二次签证，要隔多长时间才能签？ 港澳通行证-因私业务-补办港澳通行证-暂住本市的华侨（补办）
11 请问，澳门二次签证，要隔多长时间才能签？ __label__港澳通行证
12 嗨 人机互动
12 嗨 __label__UNK
13 需要的证件 港澳通行证
13 需要的证件 __label__UNK
14 你好 人机互动
texts_set:  你好
15 在 人机互动
15 在 __label__UNK
16 下午我们的外省的港澳通行证可以签证 港澳通行证-因私业务
16 下午我们的外省的港澳通行证可以签证 __label__港澳通行证
17 外省户口续签港澳通行证 港澳通行证-因私业务-首次申领港澳通行证-本省外市户籍居民（首次）
17 外省户口续签港澳通行证 __label__港澳通行证
18 L签 此节点为根节点，无场景
18 L签 __label__UNK
19 资料 失败问题回复
19 资料 __label__UNK


In [12]:
class GngJiJin:
    def __init__(self):
        
        self.name = "公积金提取"
        self.defalut_label = __LABEL__ + "公积金提取"

        self.map_label = {"公积金提取":self.defalut_label,
#                           "公积金提取-购房（含二手房）提取":"__label__公积金提取",
#                           "公积金提取-自建、大修自有住房提取":"__label__公积金提取",
#                           "公积金提取-就同一套自有住房再次提取（续期）":"_label__公积金提取",
#                           "公积金提取-加装电梯提取":"_label__公积金提取",
#                           "公积金提取-无房提取":"__label__公积金提取",
#                           "公积金提取-租房提取(限在佛山租房)":"__label__公积金提取",
#                           "公积金提取-退休注销提取":"_label__公积金提取",
#                           "公积金提取-完全丧失劳动能力并与单位解除劳动关系注销提取":"_label__公积金提取",
#                           "公积金提取-出境定居注销提取(非港澳台中国公民)":"_label__公积金提取",
#                           "公积金提取-出境定居注销提取(港澳台人员以及外国人)":"_label__公积金提取",
#                           "公积金提取-职工在职期间死亡注销提取":"_label__公积金提取",
                          "失败问题回复":__LABEL_UNK__,
                          "此节点为根节点，无场景":__LABEL_UNK__
                    }

        # 公积金提取业务相关的 关键词 （模糊，不需要精准）
        self._key = ["公积金", "提取", "无房", "租房", "自建", "自有住房", "购房", "注销", "续期"]
        
        # 可以多意图关键词
        self._key_multiple = []
        
        # 人机互动   为了获取更多语料，可以不需要
        self._key_other = ["公积金提取", "提取公积金", "无房提取", "租房提取", "自建住房提取", "大修自有住房提取", "购房提取", "购二手房提取", "加装电梯提取","再次提取", "注销提取"]
        
    def get_label(self, _class):
        _label_out = ""
        _len = len(self.name)        
        if _class[0:_len] == self.name:
            _label_out = self.defalut_label
        else:
            if _class in self.map_label:
                _label_out = self.map_label[_class]

        return _label_out

_process(base_path + '《公积金提取》对话记录.xls', "公积金提取_label.txt", GngJiJin())

2 无房提取 公积金提取-无房提取
2 无房提取 __label__公积金提取
3 无房提取要原件吗 失败问题回复
3 无房提取要原件吗 __label__UNK
4 无房提取资料 公积金提取-无房提取
4 无房提取资料 __label__公积金提取
5 公积金提取 公积金提取
5 公积金提取 __label__公积金提取
6 公积金提取 公积金提取
texts_set:  公积金提取
7 公积金提取 公积金提取
texts_set:  公积金提取
8 公积金提取所需资料 公积金提取
8 公积金提取所需资料 __label__公积金提取
9 有房提取多久可以提取一次 失败问题回复
9 有房提取多久可以提取一次 __label__UNK
10 我想了解公积金 失败问题回复
10 我想了解公积金 __label__UNK
11 公积金账号查询 失败问题回复
11 公积金账号查询 __label__UNK
12 公积金提取 公积金提取
texts_set:  公积金提取
13 怎样提取公积金 公积金提取
13 怎样提取公积金 __label__公积金提取
14 如何查询自己公积金 失败问题回复
14 如何查询自己公积金 __label__UNK
15 公积金怎么用 失败问题回复
15 公积金怎么用 __label__UNK
16 租房提取 公积金提取-租房提取(限在佛山租房)
16 租房提取 __label__公积金提取
17 无房提取 公积金提取-无房提取
texts_set:  无房提取
18 提取公积金后会影响购房贷款吗？ 失败问题回复
18 提取公积金后会影响购房贷款吗？ __label__UNK
19 住房公积金提取 我之前已经成功办理了 现在是不是说三年后又需要什么资料 公积金提取
19 住房公积金提取 我之前已经成功办理了 现在是不是说三年后又需要什么资料 __label__公积金提取


In [13]:
# _question = table1['Unnamed: 2']
# _classfilter = table1['Unnamed: 6']

# fw = open("身份证_label.txt", "w", encoding="utf-8")

# texts_set = set()
# for i, (q, c) in enumerate(zip(_question, _classfilter)):
#     if i <= 1:
#         continue
    
#     if (i < 20):
#         print(str(i) + " " + q + " " + c)
    
#     _md5_v = md5(q.encode('utf-8'))
#     if _md5_v in texts_set:
#         if (i < 20):
#             print("texts_set: ", q)
#         continue
        
#     texts_set.add(_md5_v)
    
#     _label = "_label_UNK"
#     if check_key(str(q)):
#         if c in map_label:
#             _label = map_label[c]
#         else:
#             _label = "_label__身份证业务"  # 
#     else:
#         if check_key_multiple(str(q)):
#             _label = "_label__multiple"

#     if (i < 20):
#         print(str(i) + " " + q + " " + _label)
        
#     fw.write("{}{}\n".format(q, _label))

# fw.close()
    